In [12]:
from flask import Flask, render_template, request, redirect, url_for, send_file
import os
import tensorflow as tf
from PIL import Image

app = Flask(__name__)

# Load the UNet model
model = tf.keras.models.load_model('Unet_model.keras')

# Define a function to process uploaded images
def process_image(image):
    # Preprocess the image (resize, normalize, etc.)
    # Here you should preprocess the image according to the requirements of your model
    # For example, if your model expects input images of size (256, 256, 3), you can do:
    image = image.resize((128, 128))
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    # Predict using the model
    output = model.predict(image)
    # Postprocess the output image if necessary
    # For example, convert it to PIL Image
    output_image = Image.fromarray((output[0] * 255).astype('uint8'))
    return output_image

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file:
        # Save the uploaded image
        image_path = os.path.join('uploads', file.filename)
        file.save(image_path)
        # Process the uploaded image
        input_image = Image.open(image_path)
        output_image = load_image_and_preprocess(input_image)
        # Save the output image
        output_image_path = os.path.join('static', 'output_image.jpg')
        output_image.save(output_image_path)
        # Return the output image
        return redirect(url_for('result'))
    return redirect(request.url)

@app.route('/result')
def result():
    return render_template('result.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2024 11:01:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2024 11:01:20] "GET / HTTP/1.1" 200 -
[2024-04-08 11:01:31,480] ERROR in app: Exception on /upload [POST]
Traceback (most recent call last):
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\fl

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_file
import os
import tensorflow as tf
from PIL import Image

app = Flask(__name__)

# Load the UNet model
model = tf.keras.models.load_model('Unet_model.keras')

# Define a function to process uploaded images
def process_image(image):
    # Preprocess the image (resize, normalize, etc.)
    # Here you should preprocess the image according to the requirements of your model
    # For example, if your model expects input images of size (256, 256, 3), you can do:
    image = image.resize((128, 128))
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    # Predict using the model
    output = model.predict(image)
    # Postprocess the output image if necessary
    # For example, convert it to PIL Image
    output_image = Image.fromarray((output[0] * 255).astype('uint8'))
    return output_image

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file:
        # Save the uploaded image
        image_path = os.path.join('uploads', file.filename)
        file.save(image_path)
        # Process the uploaded image
        input_image = Image.open(image_path)
        # Process the RGB image
        output_image = process_image(input_image)
        # Save the output image
        output_image_path = os.path.join('static', 'output_image.jpg')
        output_image.save(output_image_path)
        # Redirect to result page
        return redirect(url_for('result'))
    return redirect(request.url)

@app.route('/result')
def result():
    return render_template('result.html')

if __name__ == '__main__':
    app.run()

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_file
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.image import resize
from PIL import Image
import numpy as np

app = Flask(__name__)

# Load the UNet model
model = tf.keras.models.load_model('Unet_model.keras')

# Define a function to process uploaded images
def load_and_preprocess_image(image_path):
    def resize(input_image):
        input_image = tf.image.resize(input_image, (128, 128), method="nearest")
        input_image.set_shape([128, 128, 3])  # Explicitly set the shape
        return input_image

    def enhance_contrast(input_image):
        enhanced_image = tf.image.per_image_standardization(input_image)
        return enhanced_image

    def normalize(input_image):
        input_image = tf.cast(input_image, tf.float32) / 255.0
        return input_image

    input_image = tf.io.read_file(image_path)
    # input_image = tf.image.decode_png(input_image, channels=3)
    input_image = resize(input_image)
    input_image = enhance_contrast(input_image)
    input_image = normalize(input_image)
    
    return input_image


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)
    
    file = request.files['file']
    
    if file.filename == '':
        return redirect(request.url)
    
    if file:
        # Save the uploaded file temporarily
        upload_folder = 'uploads'
        file_path = os.path.join(upload_folder, file.filename)
        file.save(file_path)
        # Load and preprocess the uploaded image
        input_image = Image.open(file_path)
        processed_image = load_and_preprocess_image(file_path)
        # Convert processed image to PIL Image for saving
        # processed_image_pil = tf.keras.preprocessing.image.array_to_img(processed_image)
        # Save the output image
        output_folder = 'static'
        output_image_path = os.path.join(output_folder, 'output_image.jpg')
        processed_image_pil.save(output_image_path)
        
        # Redirect to result page
        return redirect(url_for('result'))
    
    return redirect(request.url)

@app.route('/result')
def result():
    return render_template('result.html')

if __name__ == '__main__':
    app.run()

In [9]:
from flask import Flask, render_template, request, redirect, url_for
import os
import tensorflow as tf
from PIL import Image
import cv2

app = Flask(__name__)

# Load the UNet model
model = tf.keras.models.load_model('Unet_model.keras')

# Define a function to preprocess the input image
def preprocess_image(image_path):
    # img = Image.open(image_path)
    img=cv2.imread(image_path)
    img = cv2.resize(img,(128, 128))  # Assuming you want to resize to 128x128
    img.shape
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0   # Normalize
    # img_array = np.expand_dims(img_array, axis=-1)  # Add grayscale channel
    # img_array = np.concatenate([img_array] * 3, axis=-1)  # Replicate grayscale channel to match 3 channels
    return img_array

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    if file:
        # Save the uploaded file temporarily
        upload_folder = 'uploads'
        file_path = os.path.join(upload_folder, file.filename)
        file.save(file_path)
        
        # Preprocess the uploaded image
        processed_image = preprocess_image(file_path)
        
        # Predict using the model
        predicted_image = model.predict(processed_image)
        
        # Save the predicted image
        output_folder = 'static'
        output_image_path = os.path.join(output_folder, 'output_image.jpg')
        Image.fromarray((predicted_image * 255).astype('uint8')).save(output_image_path)

        # Redirect to result page
        return redirect(url_for('result'))

    return redirect(request.url)

@app.route('/result')
def result():
    return render_template('result.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2024 10:50:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2024 10:50:25] "GET / HTTP/1.1" 200 -
[2024-04-08 10:50:34,071] ERROR in app: Exception on /upload [POST]
Traceback (most recent call last):
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Vansh Dalal\AppData\Local\Programs\Python\Python311\Lib\site-packages\fl

In [14]:
import cv2

In [18]:
i=cv2.imread('1080.png')
r=cv2.resize(i,(128,128))
r.shape

(128, 128, 3)

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import os
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

app = Flask(__name__)

# Load the UNet model
new_model = tf.keras.models.load_model('Unet_model.keras')

# Define preprocessing functions
def resize(input_image):
    input_image = tf.image.resize(input_image, (128, 128), method="nearest")
    input_image.set_shape([128, 128, 3])  # Explicitly set the shape
    return input_image

def enhance_contrast(input_image):
    enhanced_image = tf.image.per_image_standardization(input_image)
    return enhanced_image

def normalize(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image

def load_image_and_preprocess(image_path):
    input_image = tf.io.read_file(image_path)
    input_image = tf.image.decode_png(input_image, channels=3)
    input_image = resize(input_image)
    input_image = enhance_contrast(input_image)
    input_image = normalize(input_image)
    return input_image

def display_2(display_list):
    plt.figure(figsize=(10, 12))
    title = ["Input Image","Predicted_Mask"]
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]), cmap = 'gray')
        plt.axis("off")
        plt.savefig('static/pred.jpg')
    plt.show()

def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

def show_predictions(dataset=None, num=1):
  if dataset:
    for image in dataset.take(num):
      pred_mask = new_model.predict(image)
      display_2([image[0], create_mask(pred_mask)])

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/uploads', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    if file:
        # Save the uploaded file temporarily
        # Define image directory
        # image_directory = "uploads"
        
        image_path = os.path.join('uploads', file.filename)
        file.save(image_path)
        
        # Get only files from the image directory
        # image_file_paths = [os.path.join(upload_folder, filename) for filename in os.listdir(upload_folder) if os.path.isfile(os.path.join(upload_folder, filename))]

        # Create a TensorFlow dataset from the list of file paths
        image_dataset = tf.data.Dataset.from_tensor_slices(image_path)

        # Map the loading and preprocessing function to the dataset
        image_dataset = image_dataset.map(load_image_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        test_batches = image_dataset.take(1).batch(1)
        
        # Now, you can iterate through the dataset and predict
        # Take a single batch for demonstration
        show_predictions(test_batches.take(1),1)
        
        # Save the predicted mask image
        # output_folder = 'static'
        # output_image_path = os.path.join(output_folder, 'pred.jpg')
        # Image.fromarray(predicted_mask.astype('uint8')).save(output_image_path)

        # Redirect to result page
        return redirect(url_for('result'))

    return redirect(request.url)

@app.route('/')
def result():
    return render_template('result.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2024 11:08:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2024 11:08:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2024 11:09:04] "POST /upload HTTP/1.1" 404 -
127.0.0.1 - - [08/Apr/2024 11:09:04] "GET /upload HTTP/1.1" 404 -


In [ ]:
import os
import tensorflow as tf
from PIL import Image

# Define image directory
image_directory = "/content"

# Get only files from the image directory
image_file_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, filename))]

# Define preprocessing functions
def resize(input_image):
    input_image = tf.image.resize(input_image, (128, 128), method="nearest")
    input_image.set_shape([128, 128, 3])  # Explicitly set the shape
    return input_image

def enhance_contrast(input_image):
    enhanced_image = tf.image.per_image_standardization(input_image)
    return enhanced_image

def normalize(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image

def load_image_and_preprocess(image_path):
    input_image = tf.io.read_file(image_path)
    input_image = tf.image.decode_png(input_image, channels=3)
    input_image = resize(input_image)
    input_image = enhance_contrast(input_image)
    input_image = normalize(input_image)
    return input_image

def display_2(display_list):
    plt.figure(figsize=(10, 12))
    title = ["Input Image","Predicted_Mask"]
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]), cmap = 'gray')
        plt.axis("off")
        plt.savefig('mask_1.png')
    plt.show()

def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def show_predictions(dataset=None, num=1):
  if dataset:
    for image in dataset.take(num):
      pred_mask = new_model.predict(image)
      display_2([image[0], create_mask(pred_mask)])

# Create a TensorFlow dataset from the list of file paths
image_dataset = tf.data.Dataset.from_tensor_slices(image_file_paths)

# Map the loading and preprocessing function to the dataset
image_dataset = image_dataset.map(load_image_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
print(image_dataset)
test_batches = image_dataset.take(1).batch(1)

# Now, you can iterate through the dataset and predict
 # Take a single batch for demonstration
show_predictions(test_batches.take(1),1)